# Install Library

In [1]:
# Install PyTorch Scatter, PyTorch Sparse, and PyTorch Geometric
!pip install torch_geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 602.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
# Pytorch library
import torch
from torch_geometric.utils import remove_isolated_nodes
import torch.nn.functional as F
import torch
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv, RGATConv

# Dataset
from torch_geometric.datasets import Planetoid, KarateClub, Entities

# Numpy for matrices
import numpy as np

# Visualization
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import os.path as osp
import time

In [3]:
torchversion = torch.__version__

In [4]:
np.random.seed(0)

# 1. Graph Attention Networks

In [5]:
# Import dataset from PyTorch Geometric
dataset = Planetoid(root=".", name="cora")

data = dataset[0]

Processing...
Done!


## Print dataset's information

In [6]:
# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Dataset: cora()
-------------------
Number of graphs: 1
Number of nodes: 2708
Number of features: 1433
Number of classes: 7

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


In [11]:
isolated = (remove_isolated_nodes(data['edge_index'])[2] == False).sum(dim=0).item()
print(f'Number of isolated nodes = {isolated}')

Number of isolated nodes = 0


## Implement GAT
### GAT model

In [12]:
class GAT(torch.nn.Module):
  """Graph Attention Network"""
  def __init__(self, dim_in, dim_h, dim_out, heads=8):
    super().__init__()
    self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
    self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.005,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.6, training=self.training)
    h = self.gat1(x, edge_index)
    h = F.elu(h)
    h = F.dropout(h, p=0.6, training=self.training)
    h = self.gat2(h, edge_index)
    return h, F.log_softmax(h, dim=1)

### Train function

In [14]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 15

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        _, out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])

        # Print metrics every 10 epochs
        if(epoch % 1 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  f'{acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                  f'Val Acc: {val_acc*100:.2f}%')

    return model

def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

### Train GAT

In [15]:
%%time

# Create GAT model
gat = GAT(dataset.num_features, 8, dataset.num_classes)
print(gat)

GAT(
  (gat1): GATv2Conv(1433, 8, heads=8)
  (gat2): GATv2Conv(64, 7, heads=1)
)
CPU times: user 94 ms, sys: 5.63 ms, total: 99.7 ms
Wall time: 101 ms


In [16]:
# Train
train(gat, data)

# Test
acc = test(gat, data)
print(f'\nGAT test accuracy: {acc*100:.2f}%\n')

Epoch   0 | Train Loss: 1.948 | Train Acc:  18.57% | Val Loss: 1.90 | Val Acc: 32.60%
Epoch   1 | Train Loss: 1.696 | Train Acc:  80.71% | Val Loss: 1.81 | Val Acc: 52.60%
Epoch   2 | Train Loss: 1.480 | Train Acc:  92.86% | Val Loss: 1.69 | Val Acc: 64.80%
Epoch   3 | Train Loss: 1.279 | Train Acc:  97.14% | Val Loss: 1.59 | Val Acc: 69.60%
Epoch   4 | Train Loss: 1.091 | Train Acc:  95.00% | Val Loss: 1.46 | Val Acc: 73.80%
Epoch   5 | Train Loss: 0.916 | Train Acc:  96.43% | Val Loss: 1.35 | Val Acc: 75.00%
Epoch   6 | Train Loss: 0.772 | Train Acc:  98.57% | Val Loss: 1.25 | Val Acc: 76.40%
Epoch   7 | Train Loss: 0.638 | Train Acc:  97.14% | Val Loss: 1.13 | Val Acc: 78.00%
Epoch   8 | Train Loss: 0.523 | Train Acc:  98.57% | Val Loss: 1.07 | Val Acc: 75.80%
Epoch   9 | Train Loss: 0.449 | Train Acc:  98.57% | Val Loss: 1.01 | Val Acc: 77.80%
Epoch  10 | Train Loss: 0.368 | Train Acc:  97.14% | Val Loss: 0.96 | Val Acc: 76.80%
Epoch  11 | Train Loss: 0.308 | Train Acc:  98.57% | V

# GATv2

In [17]:
class GATv2(torch.nn.Module):
    def __init__(self,num_features,dims, num_classes,  drop=0.0):
        super(GATv2, self).__init__()
        heads = 8
        self.h = None
        self.conv1 = GATv2Conv(num_features,dims, heads=heads, dropout = 0.3, concat=False)
        self.conv2 = GATv2Conv(dims, num_classes, heads=heads, concat=False, dropout=0.3)
        self.drop = torch.nn.Dropout(p=drop)
        self.optimizer = torch.optim.Adam(self.parameters(),lr=0.005,weight_decay=5e-4)
    def forward(self, x, edge_index,):
        x = F.elu(self.conv1(x, edge_index))
        x = self.drop(x)
        x = self.conv2(x, edge_index)
        return x, F.log_softmax(x, dim=1)

In [18]:
%%time

# Create GAT model
gat2 = GATv2(dataset.num_features, 8, dataset.num_classes)
print(gat2)

GATv2(
  (conv1): GATv2Conv(1433, 8, heads=8)
  (conv2): GATv2Conv(8, 7, heads=8)
  (drop): Dropout(p=0.0, inplace=False)
)
CPU times: user 9.51 ms, sys: 0 ns, total: 9.51 ms
Wall time: 11.5 ms


In [19]:
# Train
train(gat2, data)

# Test
acc = test(gat2, data)
print(f'\nGAT test accuracy: {acc*100:.2f}%\n')

Epoch   0 | Train Loss: 1.950 | Train Acc:  13.57% | Val Loss: 1.94 | Val Acc: 12.00%
Epoch   1 | Train Loss: 1.934 | Train Acc:  17.86% | Val Loss: 1.93 | Val Acc: 14.60%
Epoch   2 | Train Loss: 1.922 | Train Acc:  20.00% | Val Loss: 1.92 | Val Acc: 18.80%
Epoch   3 | Train Loss: 1.904 | Train Acc:  30.71% | Val Loss: 1.92 | Val Acc: 30.60%
Epoch   4 | Train Loss: 1.887 | Train Acc:  52.14% | Val Loss: 1.91 | Val Acc: 33.00%
Epoch   5 | Train Loss: 1.872 | Train Acc:  60.71% | Val Loss: 1.90 | Val Acc: 41.80%
Epoch   6 | Train Loss: 1.851 | Train Acc:  71.43% | Val Loss: 1.89 | Val Acc: 53.20%
Epoch   7 | Train Loss: 1.832 | Train Acc:  80.00% | Val Loss: 1.88 | Val Acc: 59.00%
Epoch   8 | Train Loss: 1.806 | Train Acc:  85.71% | Val Loss: 1.87 | Val Acc: 62.60%
Epoch   9 | Train Loss: 1.784 | Train Acc:  87.14% | Val Loss: 1.86 | Val Acc: 61.20%
Epoch  10 | Train Loss: 1.756 | Train Acc:  81.43% | Val Loss: 1.85 | Val Acc: 63.60%
Epoch  11 | Train Loss: 1.732 | Train Acc:  84.29% | V

# Relational GAT (RGAT)

In [20]:
#path = osp.join(".", '..', 'data', 'Entities')'AIFB'
dataset = Entities(".", 'AIFB')
data = dataset[0]
data.x = torch.randn(data.num_nodes, 16)

Extracting ./aifb.tgz
Processing...
Done!


## RGAT model

In [21]:
class RGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,
                 num_relations):
        super().__init__()
        self.conv1 = RGATConv(in_channels, hidden_channels, num_relations)
        self.conv2 = RGATConv(hidden_channels, hidden_channels, num_relations)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_type):
        x = self.conv1(x, edge_index, edge_type).relu()
        x = self.conv2(x, edge_index, edge_type).relu()
        x = self.lin(x)
        return F.log_softmax(x, dim=-1)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = RGAT(16, 16, dataset.num_classes, dataset.num_relations).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)

In [23]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_type)
    loss = F.nll_loss(out[data.train_idx], data.train_y)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_type).argmax(dim=-1)
    train_acc = float((pred[data.train_idx] == data.train_y).float().mean())
    test_acc = float((pred[data.test_idx] == data.test_y).float().mean())
    return train_acc, test_acc

In [24]:
times = []
for epoch in range(1, 16):
    start = time.time()
    loss = train()
    train_acc, test_acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f} '
          f'Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 01, Loss: 1.4440, Train: 0.0857 Test: 0.0556
Epoch: 02, Loss: 1.3809, Train: 0.1571 Test: 0.0833
Epoch: 03, Loss: 1.3189, Train: 0.5786 Test: 0.4722
Epoch: 04, Loss: 1.2599, Train: 0.5929 Test: 0.5000
Epoch: 05, Loss: 1.1968, Train: 0.6143 Test: 0.5833
Epoch: 06, Loss: 1.1288, Train: 0.6143 Test: 0.6667
Epoch: 07, Loss: 1.0552, Train: 0.6714 Test: 0.6944
Epoch: 08, Loss: 0.9769, Train: 0.6857 Test: 0.6944
Epoch: 09, Loss: 0.8929, Train: 0.7214 Test: 0.6944
Epoch: 10, Loss: 0.8035, Train: 0.7357 Test: 0.6944
Epoch: 11, Loss: 0.7161, Train: 0.7571 Test: 0.6944
Epoch: 12, Loss: 0.6340, Train: 0.8071 Test: 0.7222
Epoch: 13, Loss: 0.5509, Train: 0.8357 Test: 0.7778
Epoch: 14, Loss: 0.4702, Train: 0.8571 Test: 0.7778
Epoch: 15, Loss: 0.4037, Train: 0.8929 Test: 0.8056
Median time per epoch: 0.8513s
